<a href="https://colab.research.google.com/github/JanePer/AI---PORTFOLIO/blob/main/Task2(Ultra_Pro)_%D0%9F%D0%BE%D0%BB%D0%BD%D0%BE%D1%81%D0%B2%D1%8F%D0%B7%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8%2C_%D0%BE%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Импорт библиотек.**

In [ ]:
from tensorflow.keras.datasets import mnist #images with numbers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential #type of NN's model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization #main layers
from tensorflow.keras.optimizers import Adam #optimizer type
from tensorflow.keras import utils #utils for to_categorical
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing # Пакет предварительной обработки данных
import numpy as np
import pandas as pd # Библиотека pandas
import pylab #for charts
import matplotlib.pyplot as plt #for charts
from mpl_toolkits.mplot3d import Axes3D #for 3D charts
from google.colab import files #for files uploading
from PIL import Image
%matplotlib inline

**2. Загрузка данных.**

In [ ]:
cars = pd.read_csv('/cars_new.csv', sep=',')
print(cars.shape)
cars.head()

(70119, 10)


,mark,model,price,year,mileage,body,kpp,fuel,volume,power
0,kia,cerato,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,nexia 1 поколение [2-й рестайлинг],140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,jimny 3 поколение [рестайлинг],750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,x1 18 e84 [рестайлинг],970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,lacetti 1 поколение,205000,2007,151445,седан,механика,бензин,1.4,95.0


**3. Подготовка данных и формирование обучающей, проверочной и тестовой выборок.**

In [ ]:
def create_dict(uniq_list):
  dictionary = {}                          # Создаём пустой словарь
  for _id, name in enumerate(uniq_list):    # Проходим по всем парам - id и название
    dictionary.update({name: _id})         # Добавляем в словарь
  return dictionary

In [ ]:
# Создаём словари по всем текстовым колонкам
marks_dict = create_dict(cars['mark'].unique())
models_dict = create_dict(set(cars['model']))
bodies_dict = create_dict(set(cars['body']))
kpps_dict = create_dict(set(cars['kpp']))
fuels_dict = create_dict(set(cars['fuel']))
marks_dict

{'audi': 13,
 'bmw': 3,
 'chery': 8,
 'chevrolet': 4,
 'daewoo': 1,
 'ford': 9,
 'honda': 19,
 'hyundai': 5,
 'kia': 0,
 'mazda': 10,
 'mercedes-benz': 7,
 'mitsubishi': 14,
 'nissan': 16,
 'opel': 15,
 'peugeot': 11,
 'renault': 20,
 'skoda': 17,
 'subaru': 18,
 'suzuki': 2,
 'toyota': 6,
 'volkswagen': 12}

In [ ]:
# Запоминаем цены
prices = np.array(cars['price'], dtype=np.float)
print(prices.shape)
prices

(70119,)


array([ 996000.,  140200.,  750000., ..., 1140000.,  550550.,  430000.])

In [ ]:
# Запоминаем числовые параметры и нормируем
years = preprocessing.scale(cars['year'])
mileages = preprocessing.scale(cars['mileage'])
volumes = preprocessing.scale(cars['volume'])
powers = preprocessing.scale(cars['power'])
print(years.std(),mileages.std(),volumes.std(),powers.std())

1.0 1.0 1.0 1.0


In [ ]:
# Создаём общие пустые выборки
x = []
y = []

# Функция преобразования в one hot encoding (ohe)
def to_ohe(value, d):
  arr = [0] * len(d)
  arr[d[value]] = 1
  return arr

In [ ]:
# Проходим по всем машинам
for _id, car in enumerate(np.array(cars)):
  # В y добавляем цену
  y.append(prices[_id])
  # В x объединяем все параметры
  # Категорийные параметры добавляем в виде ohe
  # Числовые параметры добавляем напрямую
  x_tr = to_ohe(car[0], marks_dict) + \
        to_ohe(car[1], models_dict) + \
        to_ohe(car[5], bodies_dict) + \
        to_ohe(car[6], kpps_dict) + \
        to_ohe(car[7], fuels_dict) + \
        [years[_id]] + \
        [mileages[_id]] + \
        [volumes[_id]] + \
        [powers[_id]]
  
  # Добавляем текущую строку в общий x
  x.append(x_tr)

In [ ]:
# Превращаем лист в numpy.array
x = np.array(x, dtype=np.float)
y = np.array(y, dtype=np.float)

# Выводим размеры выборок
print(x.shape)
print(y.shape)

(70119, 3208)
(70119,)


In [ ]:
# Выводим один x
print(x[0,:20])
print(x[0,-20:])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.          1.
  0.          0.          0.          0.          1.5200145  -1.40018212
  0.12288486  0.22905575]


In [ ]:
# Выводим y
print(y[:10])

[ 996000.  140200.  750000.  970000.  205000.  985000.  589000.  500000.
 1320000.  270000.]


In [ ]:
# Делаем разбивку общей выборки на обучающую и тестовую
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(63107, 3208)
(7012, 3208)
(63107,)
(7012,)


In [ ]:
# Нормализуем y_train и y_test
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1)).flatten()
y_test_scaled = y_scaler.fit_transform(y_test.reshape(-1,1)).flatten()
y_train_scaled.shape

(63107,)

In [ ]:
# Выводим базовый и нормированный y_train
print(y_train[:10])
print(y_train_scaled[:10])

[270000. 580000. 130000. 677000. 545000. 750000. 135000. 375000. 350000.
  98500.]
[-0.41387503  0.08354598 -0.63851677  0.23919062  0.02738555  0.35632525
 -0.63049385 -0.24539372 -0.28550831 -0.68906117]


**4. Создание и обучение сети.**

In [ ]:
model_ula = Sequential()
model_ula.add(Dense(200, input_dim=3208, activation='tanh'))
#model_ula.add(Dropout(0.1))
#model_ula.add(BatchNormalization())
model_ula.add(Dense(400, activation='tanh'))
#model_ula.add(Dropout(0.6))
#model_ula.add(BatchNormalization())
model_ula.add(Dense(1, activation='linear'))

model_ula.compile(loss='mse', optimizer='Adam')
model=model_ula.fit(x_train,y_train_scaled, batch_size=200, epochs=30, verbose=1, validation_split=0.1)

Epoch 1/30
284/284 [==============================] - 6s 18ms/step - loss: 0.4535 - val_loss: 0.1234
Epoch 2/30
284/284 [==============================] - 5s 17ms/step - loss: 0.2103 - val_loss: 0.1209
Epoch 3/30
284/284 [==============================] - 5s 17ms/step - loss: 0.2169 - val_loss: 0.0755
Epoch 4/30
284/284 [==============================] - 5s 17ms/step - loss: 0.1353 - val_loss: 0.1006
Epoch 5/30
284/284 [==============================] - 5s 17ms/step - loss: 0.1202 - val_loss: 0.0644
Epoch 6/30
284/284 [==============================] - 5s 17ms/step - loss: 0.0764 - val_loss: 0.0601
Epoch 7/30
284/284 [==============================] - 5s 17ms/step - loss: 0.0609 - val_loss: 0.0670
Epoch 8/30
284/284 [==============================] - 5s 17ms/step - loss: 0.0811 - val_loss: 0.0655
Epoch 9/30
284/284 [==============================] - 5s 17ms/step - loss: 0.0601 - val_loss: 0.0572
Epoch 10/30
284/284 [==============================] - 5s 17ms/step - loss: 0.0829 - val_lo

In [ ]:
# Тестим модель на тестовой выборке, чем мельше ошибка, тем точнее модель
scores=model_ula.evaluate(x_test,y_test_scaled, verbose=1)

220/220 [==============================] - 0s 2ms/step - loss: 0.0603


In [ ]:
# Используем сеть для предсказания цены - на выходе цена нормированная
predict = model_ula.predict(x_test)
predict

array([[-0.40559646],
       [ 0.1411871 ],
       [ 1.4112773 ],
       ...,
       [ 3.0444582 ],
       [ 5.860988  ],
       [-0.28571287]], dtype=float32)

In [ ]:
# Приводим предсказанные цены к первоначальному диапазону
predict_inverse = y_scaler.inverse_transform(predict).flatten()
predict_inverse

array([ 271211.84,  614508.7 , 1411932.  , ..., 2437321.  , 4205673.  ,
        346480.5 ], dtype=float32)

In [ ]:
print('средняя цена в тестовой выборке: ', round(y_test.mean(),1))
print('средняя предсказанная цена: ', round(predict_inverse.mean(),1))

средняя цена в тестовой выборке:  525864.7
средняя предсказанная цена:  544760.0


In [ ]:
# Считаем среднюю ошибку в ценах
loss_per_price=abs(predict_inverse-y_test)
print(round(loss_per_price.mean(),1),"\n", round(loss_per_price.std(),1))

71732.1 
 136518.4


In [ ]:
# Считаем средний процент ошибки
loss_per_price_perc=(loss_per_price*100)/y_test
print(round(loss_per_price_perc.mean(),1), "\n", round(loss_per_price_perc.std(),1))


27.0 
 54.5


- Модель далеко не точная, средний процент ошибки цены 27% при изменчивости аж до 55%, то есть часто предсказанная цена практически вдвое больше или меньше от реальной.